In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import folium


   # Creating Classes

   **Road data handling class**

In [2]:
class Road_Features():
    """
    This class reads the csvs of Road data , clean and returen the organized dataFrames
    """
    def __init__(self):
        self.Signal_Handling()
  
        
    
        self.Speed_Handling()
        self.Camera_Handling()
        self.Incidents_Handling()
        self.Sign_Handling()
        self.Volume_Handling()
        

    def Speed_Handling(self):
        self.speed_limit=pd.read_csv('Speed_Limits.csv')
        
        self.speed_limit.rename(columns={'STREET_NAME':'Location','SPEED':'Speed','multiline':'Coordinates','CREATED_DT':'Date'},inplace=True)
        coord=self.speed_limit['Coordinates'].str.split(expand=True)
        
        for i in range(len(coord.columns)):
            if(i>0):
                coord[i]= coord[i].str.strip(')(,')
        coord.drop(columns=0,inplace=True)
        self.speed_limit['Date']=pd.to_datetime(self.speed_limit['Date'])
        self.speed_limit['Year']=pd.DatetimeIndex(self.speed_limit['Date']).year
        self.speed_limit=self.speed_limit[['Year','Speed']]
        self.speed_limit=pd.merge(self.speed_limit,coord,left_index=True,right_index=True)
        self.speed_limit=self.speed_limit.loc[self.speed_limit['Year']<=2018]
    def Camera_Handling(self):
        self.cameras=pd.read_csv('Traffic_Camera_Locations.csv')
        self.cameras=self.cameras[['longitude','latitude']]
#         print(self.cameras)
    def Incidents_Handling(self):
        self.incidents=pd.read_csv('Traffic_Incidents.csv')
        self.incidents.rename(columns={'INCIDENT INFO':'Location','START_DT':'Date'},inplace=True)
        self.incidents['Date']=pd.to_datetime(self.incidents['Date'])
        self.incidents['Year']=pd.DatetimeIndex(self.incidents['Date']).year
        self.incidents=self.incidents[['Location','Year','Longitude','Latitude','Count']]
        self.incidents=self.incidents.loc[self.incidents['Year']==2018]
        self.incidents=self.incidents.groupby('Location').sum().reset_index()
        self.incidents['Longitude']=self.incidents['Longitude']/self.incidents['Count']
        self.incidents['Latitude']=self.incidents['Latitude']/self.incidents['Count']
        self.incidents['Year']=self.incidents['Year']/self.incidents['Count']
        self.incidents.to_csv('organzied_incidents.csv')
    def Signal_Handling(self):
        self.signals=pd.read_csv('Traffic_Signals.csv')
        self.signals.rename(columns={'INSTDATE':'Date','INT_TYPE':'Signal Type','latitude':'Latitude',\
                                     'longitude':'Longitude'},inplace=True)
        self.signals['Intersection']=self.signals['FIRSTROAD']+' & '+self.signals['SECONDROAD']
        self.signals['Date']=pd.to_datetime(self.signals['Date'])
        self.signals['Year']=pd.DatetimeIndex(self.signals['Date']).year
        self.signals['Signal Type']=self.signals['Signal Type'].astype(str)
        self.signals=self.signals[['Intersection','Signal Type','Year','Longitude','Latitude','Count']]
        
        self.signals=self.signals.loc[self.signals['Year']<=2018]
        self.signals=self.signals.groupby('Intersection').sum().reset_index()
        
        self.signals['Longitude']=self.signals['Longitude']/self.signals['Count']
        self.signals['Latitude']=self.signals['Latitude']/self.signals['Count'] 
        self.signals['Year']=self.signals['Year']/self.signals['Count'] 
        
    def Sign_Handling(self):
        self.signs=pd.read_csv('Traffic_Signs.csv')
        self.signs.rename(columns={'BLADE_TYPE':'Type','INSTDATE':'Date'},inplace=True)
        
        self.signs['Date']=pd.to_datetime(self.signs['Date'])
        self.signs['Year']=pd.DatetimeIndex(self.signs['Date']).year
        self.signs['Count']=1
        self.signs=self.signs[['Type','Year','POINT','Count']]
        self.signs=self.signs.loc[(self.signs['Type']=='Pedestrian') | (self.signs['Type']=='Yield') | (self.signs['Type']=='Stop') | (self.signs['Type']=='Bicycle / Pathway') | (self.signs['Type']=='Speed') | (self.signs['Type']=='School') | (self.signs['Type']=='Playground')]
        print(self.signs['Type'].unique())
        self.signs=self.signs.groupby('POINT').sum().reset_index()
        self.signs['Year']=self.signs['Year']/self.signs['Count']
        self.signs=self.signs.loc[self.signs['Year']<=2018]
        coord=self.signs['POINT'].str.split(expand=True)
#         
        for i in range(len(coord.columns)):
            
            if (i>0):
                coord[i]= coord[i].str.strip(',()')

        coord.drop(columns=0,inplace=True)
        self.signs.drop(columns='POINT',inplace=True)
        self.signs=pd.merge(self.signs,coord,left_index=True,right_index=True)
#         self.signs=

    def Volume_Handling(self):
        self.volume=pd.read_csv('Traffic_Volumes_for_2018.csv')
        self.volume.rename(columns={'YEAR':'Year','SECNAME':'Location','VOLUME':'Volume'},inplace=True)
        coord=self.volume['multilinestring'].str.split(expand=True)
        for i in range(len(coord.columns)):
            
            if (i>0):
                coord[i]= coord[i].str.strip(',()')
        coord.drop(columns=0,inplace=True)
        self.volume=self.volume[['Year','Location','Volume']]
        self.volume=pd.merge(self.volume,coord,left_index=True,right_index=True)
    
    def get_Speed(self):
        return self.speed_limit
    def get_Camera(self):
        return self.cameras
    def get_Incident(self):
        return self.incidents
    def get_Signal (self):
        return self.signals
    def get_Sign (self):
        return self.signs
    def get_Volume(self):
        return self.volume
        

**Weather data Class**

In [3]:
class Weather_Features():
#     def __init__(self):
#         self.download_weather_data()
        
    def download_weather_data(self):
        self.url_template = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID={station}&Year={year}&Month={month}&Day=14&timeframe={timeframe}&submit=Download+Data"
        self.url = self.url_template.format(station = 50430, year=2018, month=1, timeframe = 1)
        self.weather_data = pd.read_csv(self.url, index_col='Date/Time', parse_dates=True)
        self.weather_data.columns = [col.replace('\xb0', '') for col in self.weather_data.columns]
        self.data_df = pd.DataFrame(self.weather_data)
        self.data_df = self.data_df[['Temp (C)','Visibility (km)']].describe().reset_index()
        return self.data_df


**Map Class**

In [66]:
class Map ():
    """
    this class generates the map with boundary and gridlines, also generate the grid coordinates
    Todo- add markers for each grid to popup stats per grid
    """
    def __init__(self):
        self.boundary=pd.read_csv('City_Boundary_layer.csv')
        self.boundary=self.boundary['the_geom'].str.split(expand=True)
        
        
        for i in range(len(self.boundary.columns)):
            if(i>0):
                self.boundary[i]= self.boundary[i].str.strip(')(,')
        self.boundary.drop(columns=0,inplace=True)
        self.Boundary()
        self.draw_Boundary()
        self.draw_Grid()
        self.generate_Grid_Coord()
        
    def Boundary(self):
#         self.boundary_coordinates=[]
       
        df=pd.DataFrame(columns=['Lat','Long'])
        for i in range(0,len(self.boundary.columns),2):
            if(i+1)<len(self.boundary.columns):
                df = df.append({'Long':self.boundary.iloc[0,(i)],'Lat':self.boundary.iloc[0,(i+1)]}, ignore_index=True)
#                 tes['Long'].append(float(self.boundary.iloc[0,(i)])
#                 tes['Lat'].append(float(self.boundary.iloc[0,(i+1)])
#                 temp=[float(self.boundary.iloc[0,i+1]),float(self.boundary.iloc[0,(i)])]
#                 self.boundary_coordinates.append(temp)
            else:
                break
        self.min_lat=float(df['Lat'].min())
        self.max_lat=float(df['Lat'].max())
        self.min_long=float(df['Long'].min())
        self.max_long=float(df['Long'].max())
#         print(self.min_lat,self.max_long)
        print(self.min_lat,self.min_long)
#         print(self.max_lat,self.min_long)
        print(self.max_lat,self.max_long)
    def draw_Boundary(self):
        self.m=folium.Map(location=[51.048615,-114.070847],zoom_start=10)
        cord=[[self.min_lat,self.max_long],[self.min_lat,self.min_long],
              [self.max_lat,self.min_long],[self.max_lat,self.max_long],[self.min_lat,self.max_long]]
        
        folium.PolyLine(cord,color='blue').add_to(self.m)
#         line=folium.vector_layers.PolyLine(self.boundary_coordinates,popup='<b>boundary</b>',\
#                                      tooltip='boundary',color='blue',weight=10).add_to(f1)
        
        
#         print('hter')
#         display(m)
    def draw_Grid(self):
        self.diff_long=(self.max_long-self.min_long)/10
        self.diff_lat=(self.max_lat-self.min_lat)/10
        for i in range(10):
            cord=[[self.min_lat,(self.max_long-self.diff_long*i)],[self.max_lat,(self.max_long-self.diff_long*i)]]
            line=folium.vector_layers.PolyLine(cord,color='blue').add_to(self.m)
        for i in range(10):
            cord=[[(self.max_lat-self.diff_lat*i),self.max_long],[(self.max_lat-self.diff_lat*i),self.min_long]]
            line=folium.vector_layers.PolyLine(cord,color='blue').add_to(self.m)
        
#         folium.Marker(location=[51.0532357621796,-114.035044349223],popup='max,max',color='red').add_to(self.m)
#         folium.Marker(location=[self.min_lat,self.min_long],popup='min,min',color='red').add_to(self.m)
#         folium.Marker(location=[self.max_lat-self.diff_lat,self.max_long],popup='max-1/10,max',color='red').add_to(self.m)
#         f2.add_to(self.m)
#         folium.LayerControl().add_to(self.m)
        
    def generate_Grid_Coord(self):
        self.grid_coord=pd.DataFrame(index=['max_lat','min_lat','max_long','min_long'])
        count=0
        key=1
        while(count<=9):
            for i in range(10):
                max_lat=(self.max_lat-self.diff_lat*count)
                min_lat=(self.max_lat-self.diff_lat*(count+1))
                max_long=(self.max_long-self.diff_long*i)
                min_long=(self.max_long-self.diff_long*(i+1))
#                     a=[(self.max_lat-self.diff_lat*k),(self.max_long-self.diff_long*i)]
#                     b=[(self.max_lat-self.diff_lat*k),(self.max_long-self.diff_long*(i+1))]
#                     c=[(self.max_lat-self.diff_lat*(k+1)),(self.max_long-self.diff_long*(i+1))]
#                     d=[(self.max_lat-self.diff_lat*(k+1)),(self.max_long-self.diff_long*i)]
                    
#                 folium.Marker(location=a,popup='a {}'.format(key),color='red').add_to(self.m)
#                 folium.Marker(location=b,popup='b {}'.format(key),color='red').add_to(self.m)
#                 folium.Marker(location=c,popup='c {}'.format(key),color='red').add_to(self.m)
#                 folium.Marker(location=d,popup='d {}'.format(key),color='red').add_to(self.m)
#                 self.grid_coord[key]=[a,b,c,d]
                
                self.grid_coord[key]=[max_lat,min_lat,max_long,min_long]
        
        
                key+=1
            count+=1
        self.m.save('index.html')
    
    def heat_map():
        self.heat_map = folium.Map(location=[51.048615,-114.070847],zoom_start = 13) 

        # Ensure you're handing it floats
        volume[1] = volume[1].astype(float)
        volume[2] = volume[2].astype(float)
        volume['Volume'] = volume['Volume'].astype(float)
        volume['Volume']= volume['Volume']/volume['Volume'].mean()
        # Filter the DF for rows, then columns, then remove NaNs
        #heat_df = volume[volume['Speed_limit']=='40'] # Reducing data size so it runs faster
        #heat_df = volume[volume['Year']=='2018'] # Reducing data size so it runs faster
        heat_df = volume[[2,1,'Volume']]
        #heat_df = heat_df.dropna(axis=0, subset=[1,2])

        # List comprehension to make out list of lists
        heat_data = [[row[2], row[1],row['Volume']] for index, row in heat_df.iterrows()]

        # Plot it on the map
        folium.plugins.HeatMap(heat_data, radius = 20).add_to(heat_map)
        
    def get_Grid_Coord(self):
        return self.grid_coord
    def get_map(self):
        return self.m
        



**Data Analysis Class**

In [39]:
class Data_Analysis():
    """
    This class will find if any of the data fall in the grid
    and calculate the stats
    """
    def __init__(self,speed,camera,incident,signal,sign,volume,weather,grid_coord):
        self.speed=speed
        self.camera=camera
        self.incident=incident
        self.signal=signal
        self.sign=sign
        self.volume=volume
        self.weather=weather
        self.grid_coord=grid_coord
        ## calling functions
        self.sign_Analysis()
        self.signal_Analysis()
        self.incident_Analysis()
        self.camera_Analysis()
        self.volume_Analysis()
        self.speed_Analysis()
        self.speed_Coordinates()
        self.merge_Data()

#             print()
#         self.speed.to_csv('test.csv')
#         print(len(self.speed.columns))
    def speed_Coordinates(self):
        self.speed_cord=pd.DataFrame(columns=['Speed','Coord'])
        for index,row in self.speed.iterrows():
            temp=[[]]
            for i in range(3,len(self.speed.columns)-1,2):
                if( row[i]!=None):
                    long=float(row[i])
                    lat=float(row[i+1])
                    temp.append([lat,long])
            temp.pop(0)
            self.speed_cord=self.speed_cord.append({'Speed':row['Speed'],'Coord':temp},ignore_index=True)
#         self.m=folium.Map(location=[51.048615,-114.070847],zoom_start=10)
#         folium.PolyLine(self.speed_cord.iloc[60,1],color='blue').add_to(self.m)
#         self.m.save('test.html')
                   
        
    def speed_Analysis(self):
        self.speed_per_grid=pd.DataFrame(columns=['Speed','Grid_Location','Latitude','Longitude'])
       
        
        for index,row in self.speed.iterrows():
#             temp=[[]]
            
#             print("Row ",index,"Speed",row['Speed'])
            for i in range(1,len(self.speed.columns)-1,2):
                if( row[i]!=None):
                    long=float(row[i])
                    lat=float(row[i+1])
                   
                    for k in range(1,len(self.grid_coord.columns)):
                        max_lat=float(self.grid_coord.iloc[0,k])
                        min_lat=float(self.grid_coord.iloc[1,k])
                        max_long=float(self.grid_coord.iloc[2,k])
                        min_long=float(self.grid_coord.iloc[3,k])
                        #NOTE: long logic is in reverse
                        if(min_lat<lat and lat<max_lat) and (max_long<long and long<min_long):
                            self.speed_per_grid=self.speed_per_grid.append({'Speed':row['Speed'],'Grid_Location':k,'Latitude':(max_lat+min_lat)/2,'Longitude':(max_long+min_long)/2},ignore_index=True)
        self.speed_per_grid=self.speed_per_grid.astype(int)
        self.speed_per_grid=self.speed_per_grid.groupby('Grid_Location').mean().reset_index()
        
    def volume_Analysis(self):
        self.volume_per_grid=pd.DataFrame(columns=['Volume','Grid_Location','Latitude','Longitude'])
       
        print(self.volume.columns)
        for index,row in self.volume.iterrows():
            
            for i in range(1,len(self.volume.columns)-2,2):#-2 because we have one more addition col in volume df
                if( row[i]!=None):
                    long=float(row[i])
                    lat=float(row[i+1])
                    for k in range(1,len(self.grid_coord.columns)):
                        max_lat=float(self.grid_coord.iloc[0,k])
                        min_lat=float(self.grid_coord.iloc[1,k])
                        max_long=float(self.grid_coord.iloc[2,k])
                        min_long=float(self.grid_coord.iloc[3,k])
                        #NOTE: long logic is in reverse
                        if(min_lat<lat and lat<max_lat) and (max_long<long and long<min_long):
                            self.volume_per_grid=self.volume_per_grid.append({'Volume':row['Volume'],'Grid_Location':k,'Latitude':(max_lat+min_lat)/2,'Longitude':(max_long+min_long)/2},ignore_index=True)
                else:
                    continue
        self.volume_per_grid=self.volume_per_grid.astype(int)
        self.volume_per_grid=self.volume_per_grid.groupby('Grid_Location').mean().reset_index()
#         print(self.volume_per_grid)
    def camera_Analysis (self):
        self.camera_per_grid=pd.DataFrame(columns=['Camera','Grid_Location','Latitude','Longitude'])
        for index,row in self.camera.iterrows():
#             print(row)
            long=float(row['longitude'])
            lat=float(row['latitude'])
            for k in range(1,len(self.grid_coord.columns)):
                max_lat=float(self.grid_coord.iloc[0,k])
                min_lat=float(self.grid_coord.iloc[1,k])
                max_long=float(self.grid_coord.iloc[2,k]) 
                min_long=float(self.grid_coord.iloc[3,k])
                if(min_lat<lat and lat<max_lat) and (max_long<long and long<min_long):
                    self.camera_per_grid=self.camera_per_grid.append({'Camera':1,'Grid_Location':k,'Latitude':(max_lat+min_lat)/2,'Longitude':(max_long+min_long)/2},ignore_index=True)
        self.camera_per_grid=self.camera_per_grid.astype(int)
        self.camera_per_grid=self.camera_per_grid.groupby('Grid_Location').sum().reset_index()
#         print(self.camera_per_grid)
    def incident_Analysis(self):
        self.incident_per_grid=pd.DataFrame(columns=['Incident','Grid_Location','Latitude','Longitude'])
        for index,row in self.incident.iterrows():
#             print(row)
            long=float(row['Longitude'])
            lat=float(row['Latitude'])
            for k in range(1,len(self.grid_coord.columns)):
                max_lat=float(self.grid_coord.iloc[0,k])
                min_lat=float(self.grid_coord.iloc[1,k])
                max_long=float(self.grid_coord.iloc[2,k]) 
                min_long=float(self.grid_coord.iloc[3,k])
                if(min_lat<lat and lat<max_lat) and (max_long<long and long<min_long):
                    self.incident_per_grid=self.incident_per_grid.append({'Incident':row['Count'],'Grid_Location':k,'Latitude':(max_lat+min_lat)/2,'Longitude':(max_long+min_long)/2},ignore_index=True)
        self.incident_per_grid=self.incident_per_grid.astype(int)
        self.incident_per_grid=self.incident_per_grid.groupby('Grid_Location').mean().reset_index()
#         print(self.camera_per_grid)
#         print(self.incident_per_grid)
    def signal_Analysis(self):
        self.signal_per_grid=pd.DataFrame(columns=['Signal','Grid_Location','Latitude','Longitude'])
        for index,row in self.signal.iterrows():
#             print(row)
            long=float(row['Longitude'])
            lat=float(row['Latitude'])
            for k in range(1,len(self.grid_coord.columns)):
                max_lat=float(self.grid_coord.iloc[0,k])
                min_lat=float(self.grid_coord.iloc[1,k])
                max_long=float(self.grid_coord.iloc[2,k]) 
                min_long=float(self.grid_coord.iloc[3,k])
                if(min_lat<lat and lat<max_lat) and (max_long<long and long<min_long):
                    self.signal_per_grid=self.signal_per_grid.append({'Signal':row['Count'],'Grid_Location':k,'Latitude':(max_lat+min_lat)/2,'Longitude':(max_long+min_long)/2},ignore_index=True)
        self.signal_per_grid=self.signal_per_grid.astype(int)
        self.signal_per_grid=self.signal_per_grid.groupby('Grid_Location').sum().reset_index()
#         print(self.signal_per_grid)
    def sign_Analysis(self):
        self.sign_per_grid=pd.DataFrame(columns=['Sign','Grid_Location','Latitude','Longitude'])
        for index,row in self.sign.iterrows():
#             print(row)
            long=float(row[1])
            lat=float(row[2])
            for k in range(1,len(self.grid_coord.columns)):
                max_lat=float(self.grid_coord.iloc[0,k])
                min_lat=float(self.grid_coord.iloc[1,k])
                max_long=float(self.grid_coord.iloc[2,k]) 
                min_long=float(self.grid_coord.iloc[3,k])
                if(min_lat<lat and lat<max_lat) and (max_long<long and long<min_long):
                    self.sign_per_grid=self.sign_per_grid.append({'Sign':row['Count'],'Grid_Location':k,'Latitude':(max_lat+min_lat)/2,'Longitude':(max_long+min_long)/2},ignore_index=True)
        self.sign_per_grid=self.sign_per_grid.astype(int)
        self.sign_per_grid=self.sign_per_grid.groupby('Grid_Location').sum().reset_index()
#         print(self.sign_per_grid)
    def merge_Data(self):
#         print(self.weather.iloc[1,1]," vis:",self.weather.iloc[1,2])
        ##merged camera & signal
        df1=pd.merge(self.camera_per_grid,self.signal_per_grid,how='outer',on=['Grid_Location'])
        ## merged speed & volume
        df2=pd.merge(self.speed_per_grid,self.volume_per_grid,how='outer',on=['Grid_Location'])
        ## merged incident & sign
        df3=pd.merge(self.incident_per_grid,self.sign_per_grid,how='outer',on=['Grid_Location'])
        ## mege df1 and df2
        df4=pd.merge(df1,df2,how='outer',on=['Grid_Location'])
        ## final merge df4 and df 3
        self.stats_per_grid=pd.merge(df4,df3,how='outer',on=['Grid_Location'])
        self.stats_per_grid['Temp (C)']=self.weather.iloc[1,1]
        self.stats_per_grid['Visibility (km)']=self.weather.iloc[1,2]
#         print(df1)
    def get_Stats_per_Grid(self):
        return self.stats_per_grid
    def get_Speed_Coord (self):
        return self.speed_cord
        
        
        
        


In [82]:
class Display_Stats():
    def __init__(self,map,data,speed_cord):
        self.map=map
        self.data=data
        self.speed_cord=speed_cord
        self.draw_Speed_Map()
#         self.draw_Stats_Map()
    def draw_Stats_Map(self):
        for index,row in self.data.iterrows():
            folium.Marker(location=[row['Latitude'],row['Longitude']],popup="Camera: {} \n Signal: {} \n Speed: {} \n Volume: {} \n Incident: {} \n Sign: {} \n Temp: {} \n Visibility: {}".format(row['Camera'],row['Signal'],row['Speed'],row['Volume'],row['Incident'],row['Sign'],row['Temp (C)'],row['Visibility (km)'])).add_to(self.map)
#             print("Camera: {} \n Signal: {} \n Speed: {} \n Volume: {} \n Incident: {} \n Sign: {} \n Temp: {} \n Visibility: {}".format(row['Camera'],row['Signal'],row['Speed'],row['Volume'],row['Incident'],row['Sign'],row['Temp (C)'],row['Visibility (km)']))
#             break
        self.map.save('test.html')
    def draw_Speed_Map(self):
        self.speed_Map=folium.Map(location=[51.048615,-114.070847],zoom_start=10)
        lis=list(self.speed_cord['Speed'].unique())
        color=['gray', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige','red','pink','black']
        combo={}
        for i in range(len(lis)):
            combo[lis[i]]=color[i]
        for index,row in self.speed_cord.iterrows():
            folium.PolyLine(row['Coord'],color=combo[row['Speed']]).add_to(self.speed_Map)
        self.speed_Map.save('Speed_Map.html')
            
#         print(combo)

<Todo:>

**Step 2**

- visualize data of the merged df
- get the speed coordinated DF to plot on map (code is implemented in Data_Analysis Class)
- draw the traffic volume heat map
- if you have time plot stats/grid on map




**Main Code**

In [6]:
road=Road_Features()
speed=road.get_Speed()
camera=road.get_Camera()
incident=road.get_Incident()
signal=road.get_Signal()
sign=road.get_Sign()
volume=road.get_Volume()
weather=Weather_Features().download_weather_data()


['Pedestrian' 'Yield' 'Stop' 'Bicycle / Pathway' 'Playground' 'Speed'
 'School']


In [67]:
map=Map()

50.842822 -113.859905
51.212425 -114.315796


In [68]:
m=map.get_map()

In [8]:
grid_coord=map.get_Grid_Coord()


In [9]:
print(grid_coord.head())

                 1           2           3           4           5    \
max_lat    51.212425   51.212425   51.212425   51.212425   51.212425   
min_lat    51.175465   51.175465   51.175465   51.175465   51.175465   
max_long -114.315796 -114.270207 -114.224618 -114.179029 -114.133440   
min_long -114.270207 -114.224618 -114.179029 -114.133440 -114.087851   

                 6           7           8           9           10   ...  \
max_lat    51.212425   51.212425   51.212425   51.212425   51.212425  ...   
min_lat    51.175465   51.175465   51.175465   51.175465   51.175465  ...   
max_long -114.087851 -114.042261 -113.996672 -113.951083 -113.905494  ...   
min_long -114.042261 -113.996672 -113.951083 -113.905494 -113.859905  ...   

                 91          92          93          94          95   \
max_lat    50.879782   50.879782   50.879782   50.879782   50.879782   
min_lat    50.842822   50.842822   50.842822   50.842822   50.842822   
max_long -114.315796 -114.270207 -114

In [10]:
print(sign.loc[sign['Count']>1])

              Year  Count                  1                2
36     2012.000000      2  -113.883366768892  50.921512982233
47     1004.500000      2  -113.888549941574  51.038846768305
49     1008.000000      2  -113.888562491308  51.042849287861
54     2014.000000      2   -113.88870479289  50.970214835797
55     1004.000000      2  -113.888728082783  51.042854281209
...            ...    ...                ...              ...
43065  2016.000000      3   -114.26253893505  51.086670845272
43097  2006.000000      2  -114.265419159398  51.096386127401
43159  1338.666667      3  -114.280520938733  51.125159421598
43160  2004.000000      2   -114.28058352071  51.124963785639
43162  2004.000000      2   -114.28079398816   51.12518901914

[6938 rows x 4 columns]


In [33]:
dataAnalysis=Data_Analysis(speed,camera,incident,signal,sign,volume,weather,grid_coord)

Index([    'Year', 'Location',   'Volume',          1,          2,          3,
                4,          5,          6,          7,
       ...
              479,        480,        481,        482,        483,        484,
              485,        486,        487,        488],
      dtype='object', length=491)


In [50]:
stats_per_grid=dataAnalysis.get_Stats_per_Grid()
stats_per_grid=stats_per_grid.replace(np.NaN,0)
speed_cord=dataAnalysis.get_Speed_Coord()


In [52]:
print(stats_per_grid.sort_values('Grid_Location').describe())

       Grid_Location     Camera      Signal      Speed        Volume  \
count      82.000000  82.000000   82.000000  82.000000     82.000000   
mean       47.426829   1.536585   17.817073  53.659218  20465.026497   
std        28.553623   3.031502   28.693841  26.982176  13394.154623   
min         1.000000   0.000000    0.000000   0.000000      0.000000   
25%        23.250000   0.000000    0.000000  47.311705  12302.342139   
50%        45.500000   0.000000    9.000000  62.122950  19306.504231   
75%        72.000000   2.000000   25.750000  70.524034  28308.070553   
max        99.000000  22.000000  218.000000  98.833333  68347.826087   

        Incident         Sign      Temp (C)  Visibility (km)  
count  82.000000    82.000000  8.200000e+01     8.200000e+01  
mean    1.186535   612.841463 -6.117362e+00     3.060659e+01  
std     0.562050   633.370768  4.468221e-15     1.429831e-14  
min     0.000000     0.000000 -6.117362e+00     3.060659e+01  
25%     1.017857    46.250000 -6.117

In [79]:
print(speed_cord.head())

  Speed                                              Coord
0    40  [[50.996020318821, -114.034870203979], [50.995...
1    60  [[51.166674574918, -114.068594322365], [51.166...
2    80  [[51.062637243694, -113.918657276573], [51.062...
3    40  [[50.90171256435, -113.975005162358], [50.9017...
4    60  [[51.12605722446, -114.048743773461], [51.1260...


In [13]:
from folium.plugins import HeatMap

In [14]:
heat_map = folium.Map(location=[51.048615,-114.070847],
                    zoom_start = 13) 

# Ensure you're handing it floats
volume[1] = volume[1].astype(float)
volume[2] = volume[2].astype(float)
volume['Volume'] = volume['Volume'].astype(float)
#volume['Volume']= volume['Volume']/volume['Volume'].mean()
# Filter the DF for rows, then columns, then remove NaNs
#heat_df = volume[volume['Speed_limit']=='40'] # Reducing data size so it runs faster
#heat_df = volume[volume['Year']=='2018'] # Reducing data size so it runs faster
heat_df = volume[[2,1,'Volume']]
#heat_df = heat_df.dropna(axis=0, subset=[1,2])

# List comprehension to make out list of lists
heat_data = [[row[2], row[1],row['Volume']] for index, row in heat_df.iterrows()]

# Plot it on the map
folium.plugins.HeatMap(heat_data, radius = 20).add_to(heat_map)

#folium.LayerControl().add_to(map_hooray)

# Display the map
heat_map

In [83]:
draw_stats=Display_Stats(m,stats_per_grid,speed_cord)